<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W5/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Flatten
#from keras.layers import LSTM
#from keras.layers import InputLayer
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn import model_selection
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device=torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

data=pd.read_csv("xaa",encoding="utf-8")

vect  = CountVectorizer()
#corpus = vect.fit_transform(data["text"])
#train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data["label"],test_size=0.4)
#Encoder = LabelEncoder()
#train_label = Encoder.fit_transform(train_label)
#test_label = Encoder.fit_transform(test_label)
#train_corpus_vect=vect.transform(train_corpus)
#test_corpus_vect=vect.transform(test_corpus)

# Initalise vect.vocabulary_
vect.fit_transform(data["text"])

maxlen=0

def transform(text,vect):
    global maxlen
    d=vect.vocabulary_
    p=vect.build_preprocessor()
    t=vect.build_tokenizer()
    vec_list=[]
    for doc in text:
        tokens=t(p(doc))
        s=len(tokens)
        if s>maxlen:
            maxlen=s
        doc_vec=np.array([d[token] for token in tokens])
        #doc_vec=sequence.pad_sequences(doc_vec,maxlen=maxlen)
        vec_list.append(doc_vec)
    vec_list=sequence.pad_sequences(vec_list,maxlen=maxlen)
    corpus_vec=np.vstack(vec_list)
    return torch.tensor(corpus_vec).float()

# print(corpus_vec)

bsize=50
epochs=3
lr=1e-3

class corpus(Dataset):
    def __init__(self,corpus,label,seq):
        self.corpus=corpus
        self.label=label
        self.seq=seq
    def __len__(self):
        return len(self.corpus)
    def __getitem__(self,idx):
        return self.corpus[idx],self.label[idx]

class lstm(nn.Module):
    def __init__(self,input_size,hidden_size,seq):
        super(lstm,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.seq=seq
        self.lstm=nn.LSTM(input_size,hidden_size,batch_first=True)
        self.fc=nn.Linear(self.hidden_size*seq,1)

    def forward(self,x):
        x, (_,_) = self.lstm(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return nn.Sigmoid()(x)

Encoder = LabelEncoder()
corpus_vec = transform(data["text"],vect)
train_corpus_vec, test_corpus_vec, train_label, test_label = model_selection.train_test_split(corpus_vec,data["label"],test_size=0.4)
train_label = torch.from_numpy(Encoder.fit_transform(train_label)).float()
test_label = torch.from_numpy(Encoder.fit_transform(test_label)).float()
#train_corpus_vec = transform(train_corpus,vect)
#test_corpus_vec = transform(test_corpus,vect)
lstm_classifier=lstm(1,100,maxlen)
loss_fn=nn.BCELoss()
optimizer=torch.optim.Adam(lstm_classifier.parameters(),lr=lr)
#print(len(lstm_classifier(torch.reshape(train_corpus_vec[0],(bsize,maxlen,1)))))

train_dataloader=DataLoader(corpus(train_corpus_vec,train_label,maxlen),batch_size=bsize,shuffle=True)
test_dataloader=DataLoader(corpus(test_corpus_vec,test_label,maxlen),batch_size=bsize,shuffle=True)

def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y=x.to(device),y.to(device)
        #print(torch.sum(x[0]!=0))
        pred=model(x.reshape(bsize,dataloader.dataset.seq,1).to(device))
        #pred=model(x)
        cost=loss_fn(pred.flatten(),y)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 10 == 0:
            cost, current = cost.item(), batch * bsize + len(x)
            print(f"cost: {cost:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x,y=x.to(device),y.to(device)
            pred=model(x.reshape(bsize,dataloader.dataset.seq,1).to(device))
            #pred=model(x)
            #print(torch.round(pred.flatten()),y)
            test_loss += loss_fn(pred.flatten(), y).item()
            ncorrect = (torch.round(pred.flatten()) == y).sum().item()
            print(ncorrect)
            correct += ncorrect

    print(correct,size)
    print(f"Test Error: \n Accuracy: {(100*correct/size):>0.1f}%, Avg loss: {test_loss/size:>8f} \n")

for epoch in range(epochs):
    train(train_dataloader,lstm_classifier,loss_fn,optimizer)
test_loop(test_dataloader,lstm_classifier,loss_fn)

cost: 0.687257  [   50/12000]
cost: 0.762855  [  550/12000]
cost: 0.677522  [ 1050/12000]
cost: 0.694661  [ 1550/12000]
cost: 0.678861  [ 2050/12000]
cost: 0.685298  [ 2550/12000]
cost: 0.644602  [ 3050/12000]
cost: 0.875158  [ 3550/12000]
cost: 0.693387  [ 4050/12000]
cost: 0.749722  [ 4550/12000]
cost: 0.722870  [ 5050/12000]
cost: 0.702108  [ 5550/12000]
cost: 0.763407  [ 6050/12000]
cost: 0.765638  [ 6550/12000]
cost: 0.703042  [ 7050/12000]
cost: 0.682852  [ 7550/12000]
cost: 0.688694  [ 8050/12000]
cost: 0.776498  [ 8550/12000]
cost: 0.686863  [ 9050/12000]
cost: 0.694928  [ 9550/12000]
cost: 0.715820  [10050/12000]
cost: 0.703266  [10550/12000]
cost: 0.684015  [11050/12000]
cost: 0.699546  [11550/12000]
cost: 0.722529  [   50/12000]
cost: 0.675984  [  550/12000]
cost: 0.733863  [ 1050/12000]
cost: 0.679385  [ 1550/12000]
cost: 0.707996  [ 2050/12000]
cost: 0.713310  [ 2550/12000]
cost: 0.683178  [ 3050/12000]
cost: 0.683671  [ 3550/12000]
cost: 0.667226  [ 4050/12000]
cost: 0.68

In [6]:
!pip install keras